Для разделения на предложения используйте библиотеку razdel.

In [1]:
%pip install razdel

You should consider upgrading via the 'C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
def collecting_data():
    
    f = open('data_1.txt', 'w', encoding = "UTF-8")
    for i in range(1,6):
        tmp_data = []
        print(i)
        tmp_f = open(str(i) + ".txt", "r", encoding = "UTF-8")
        tmp_data = tmp_f.read()
        
        f.write(tmp_data)
        tmp_f.close()
        
        
collecting_data()

1
2
3
4
5


In [3]:
from razdel import sentenize

text = open("data_1.txt", "r", encoding = "UTF-8").read()

sent = list(sentenize(text))
sent_main = sent
print(sent[0])

Substring(0, 172, 'Глобальная нехватка микросхем в 2020—2022 годах (также — дефицит / кризис полупроводников) — кризис, в котором спрос на интегральные схемы превышает предложение на 10-30 %.')


In [4]:
print(sent[100].text)
print(len(sent))

Война привела к огромным экономическим и социальным потрясениям, оставив после себя разрушенную экономику, а также огромные заминированные площади земли.
325


In [5]:
import pymorphy2
import nltk
morph = pymorphy2.MorphAnalyzer(lang = 'ru')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words("russian")
#удаление стоп слов
def filtered_words(tokens, stop_words):
    filtered_tokens = []
    for token in tokens:
        if token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens


def norm_sent(sent):
    norm_text = list()
    for token in sent:
        # print(token)
        text = token.text
        # print(text)
        norm_sent = []
        
        # удаление знаков препинания
        marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_—»«'''
        for x in text:  
            if x in marks:  
                text = text.replace(x, "")
        
        # разбиение на слова
        text = word_tokenize(text, language="russian")
        # print(text)
        
        # удаление стоп-слов
        text = filtered_words(text, stop_words)
        
        for word in text:
            norm_sent.append(morph.parse(word)[0].normal_form)
        norm_text.append(norm_sent)
    
    return norm_text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
data = norm_sent(sent)
# print(data)

***Необходимо по запросам выдать наиболее подходящие предложения по запросам:***

Глобальная нехватка  чипов может продлиться от двух до четырёх лет.

Племена Эритреи общались между собой на «упрощённом итальянском».

Экономические санкции  стали инструментом геополитики ещё в V веке до н. э.



***По tf.idf***

• df в данном случае – это количество предложений, в которых
встречалось слово

• Tf (слайд 45) д.б. реализованы два варианта=

– 1) (natural) это количество упоминаний слова в предложении
(count) и

– 2) ( augmented) 0.5+0.5 (count/count_max) при count>0, =0
при count=0


– Нормализация запроса и предложения

– Программа должна выстроить все предложения из всех
статей по мере сходства с запросом по 2 векторным
моделям.

– В отчете должны быть показаны веса выдаваемых
предложений

In [7]:
N = len(data)
print(N)
print(data[1])

325
['кризис', 'затронуть', '169', 'отрасль', 'который', 'требоваться', 'полупроводник', 'включая', 'производство', 'видеокарта', 'смартфон', 'игровой', 'приставка', 'автопромышленность']


***Языковая модель Lambda = 0.5, 0.9***

В data содердатся нормализованные предложения, по которым высчитывать меру

In [8]:
def norm_Q(Q):
    Q1 = Q
    # удаление знаков препинания
    marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_—»«'''
    for x in Q1:  
        if x in marks:  
            Q1 = Q1.replace(x, "")
            
    Q1 = word_tokenize(Q1, language="russian")
    Q1 = filtered_words(Q1, stop_words)
    
    Q1_norm = []
    for word in Q1:
        Q1_norm.append(morph.parse(word)[0].normal_form)


    print(Q1_norm)
    return(Q1_norm)

In [9]:
Q1_main = 'Кризис микрочипов может продлиться от двух до 4 лет.'
Q2_main = 'Племена Эритреи общались между собой на «упрощённом итальянском».'
Q3_main = 'Экономические санкции стали инструментом геополитики ещё в V веке до н. э.'

In [10]:
Q1_norm = norm_Q(Q1_main)
Q2_norm = norm_Q(Q2_main)
Q3_norm = norm_Q(Q3_main)

['кризис', 'микрочип', 'продлиться', 'два', '4', 'год']
['племя', 'эритрея', 'общаться', 'себя', 'упрощённый', 'итальянский']
['экономический', 'санкция', 'стать', 'инструмент', 'геополитика', 'ещё', 'v', 'век', 'н', 'э']


In [11]:
len_data = 0
for sent in data:
    len_data += len(sent)
    
print(len_data)
    
    
    
def tf_word(Q_word, data):
    tf_natural = 0
    for sent in data:
        for word in sent:
            if (Q_word == word):
                tf_natural += 1
                
    return tf_natural

def tf_word_in_D(Q_word, D):
    tf_natural = 0
    for word in D:
        if (Q_word == word):
            tf_natural += 1
    return tf_natural

print(tf_word('кризис', data))

def QxD(Q, data, L):
    w_list = [0] * len(data)
    idx = 0
    
    for d in data:
        w = 1
        for word in Q:
            w_tmp = ( (1 - L) * (tf_word(word, data) / len_data) + L * (tf_word_in_D(word, d) / len(d)) )
            if (w_tmp != 0):
                w *= w_tmp
            # print(w)
        w_list[idx] = w
        idx += 1
    
    print(idx)
    # print(w_list[0:10])
    output_list = []
    idx = 0
    for val in w_list:
        if val != 0:
            output_list.append([val, idx])
        idx += 1
        
    output_Q = sorted(output_list, reverse = True)
    return output_Q

4215
6


In [31]:
Q1_out = QxD(Q1_norm, data, 0.5)
Q2_out = QxD(Q2_norm, data, 0.5)
Q3_out = QxD(Q3_norm, data, 0.5)

325
325
325


In [36]:
print(Q3_out[0:10])

[[2.8890944378732115e-19, 290], [6.846618854807974e-20, 263], [5.433652790956709e-20, 259], [4.524205651799261e-20, 269], [3.097072124503353e-20, 296], [3.097072124503353e-20, 270], [1.7728191532563637e-20, 204], [8.155797779462559e-21, 239], [7.008873529817023e-21, 262], [4.535293551928628e-21, 229]]


In [33]:
Q1_f_out = open('Q1_out_lan.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q1_main)
Q1_f_out.write('Запрос: ' + Q1_main + '\n' + 'Выдача ограничена весом более 2.1e-12' + '\n' + '\n')

for var in Q1_out:
    if (var[0] >= 2.1e-12):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q1_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q1_f_out.close()

Запрос:  Кризис микрочипов может продлиться от двух до 4 лет.


In [34]:
Q2_f_out = open('Q2_out_lan.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q2_main)
Q2_f_out.write('Запрос: ' + Q2_main + '\n' + 'Выдача ограничена весом более 2.1e-12' + '\n' + '\n')

for var in Q2_out:
    if (var[0] >= 2.1e-12):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q2_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q2_f_out.close()

Запрос:  Племена Эритреи общались между собой на «упрощённом итальянском».


In [39]:
Q3_f_out = open('Q3_out_lan.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q3_main)
Q3_f_out.write('Запрос: ' + Q3_main + '\n' + 'Выдача ограничена весом более 2.1e-21' + '\n' + '\n')

for var in Q3_out:
    if (var[0] >= 2.1e-21):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q3_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q3_f_out.close()

Запрос:  Экономические санкции стали инструментом геополитики ещё в V веке до н. э.


In [12]:
Q1_out_9 = QxD(Q1_norm, data, 0.9)
Q2_out_9 = QxD(Q2_norm, data, 0.9)
Q3_out_9 = QxD(Q3_norm, data, 0.9)

325
325
325


In [20]:
print(Q3_out_9[0:10])

[[2.930482873574492e-22, 290], [6.87682631048161e-23, 263], [5.436258084312099e-23, 259], [4.497011709806815e-23, 269], [3.062481226996224e-23, 296], [3.062481226996224e-23, 270], [1.7260882296545686e-23, 204], [7.702149285802706e-24, 239], [6.569314351842864e-24, 262], [5.202725984822126e-25, 229]]


In [16]:
Q1_f_out = open('Q1_out_lan_9.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q1_main)
Q1_f_out.write('Запрос: ' + Q1_main + '\n' + 'Выдача ограничена весом более 2.1e-14' + '\n' + '\n')

for var in Q1_out_9:
    if (var[0] >= 2.1e-14):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q1_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q1_f_out.close()

Запрос:  Кризис микрочипов может продлиться от двух до 4 лет.


In [19]:
Q2_f_out = open('Q2_out_lan_9.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q2_main)
Q2_f_out.write('Запрос: ' + Q2_main + '\n' + 'Выдача ограничена весом более 2.1e-12' + '\n' + '\n')

for var in Q2_out_9:
    if (var[0] >= 2.1e-12):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q2_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q2_f_out.close()

Запрос:  Племена Эритреи общались между собой на «упрощённом итальянском».


In [21]:
Q3_f_out = open('Q3_out_lan_9.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q3_main)
Q3_f_out.write('Запрос: ' + Q3_main + '\n' + 'Выдача ограничена весом более 2.1e-25' + '\n' + '\n')

for var in Q3_out_9:
    if (var[0] >= 2.1e-25):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q3_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q3_f_out.close()

Запрос:  Экономические санкции стали инструментом геополитики ещё в V веке до н. э.


In [7]:
# список всех уникальных лемм
unique = []
for sent in data:
    for word in sent:
        if word not in unique:
            unique.append(word)
            
# print(unique)

In [8]:
print(unique[200:215])

['рост', 'сектор', 'майнинга', 'криптовалюта', 'потреблять', 'значительный', 'высокопроизводительный', 'процессор', 'с', 'сторона', 'замедлять', 'мощность', '121', 'твтчас', 'электроэнергия']


In [9]:
len(unique)

2083

In [10]:
import math


def df_word(word, data):
    cnt = 0
    for sent in data:
        for words in sent:
            if (word == words):
                cnt += 1
                break
    return cnt

def idf_word(df, N):
    return math.log10( N / df )

В коллекции 325 документов (N), 2083 уникальных слов. DF - количество документов, в которых есть слово, IDF - log(N / DF), IDF в пределах от 2.511 до 0.64. TF в первом случае для каждого документа - сколько раз слово встречается (0-3 обычно).

Предложения длинные, после нормализации значения TF-IDF в среднем 0.1.

Из второго запроса 4 слова в базисе, после нормализации TF-IDF от 0.01 до 0.3.

Возьмем усредненное значение TF-IDF в запросе (0.25) и среднее значения TF-IDF в документах (0.1), скалярное произведение будет 0.1. В файле с выводом самый точный вывод имеет вес 0.05, с учетом того, что рассчеты усредненные, выглядит будто ошибки в вычислениях нет. 

In [11]:
idf_list = []
for word in unique:
    n1 = df_word(word, data)
    n2 = idf_word(n1, N)
    idf_list.append(n2)

print(idf_list[0:10])

[2.5118833609788744, 1.5118833609788744, 1.4327021149312495, 2.5118833609788744, 0.6485605008584184, 1.3657553253006365, 1.8129133566428555, 1.8129133566428555, 1.5576408515395495, 0.9678153166285988]


In [12]:
print(max(idf_list))

2.5118833609788744


In [13]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns = np.arange(325), index = unique)

In [14]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
глобальный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
нехватка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
микросхема,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20202022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for i in df.columns:
    tmp = data[i]
    # print("tmp: ", tmp)
    elem_id = 0
    for elem in unique:
        tf_natural = 0
        for word in tmp:
            if (elem == word):
                tf_natural += 1
        df.loc[elem, i] = tf_natural * idf_list[elem_id]
        elem_id += 1

In [16]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
глобальный,2.511883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
нехватка,1.511883,0.0,1.511883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
микросхема,1.432702,0.0,0.0,0.0,1.432702,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20202022,2.511883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
год,0.648561,0.0,0.0,0.0,1.297121,0.0,0.648561,1.297121,0.0,0.648561,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df.loc[:, 20:30]

,20,21,22,23,24,25,26,27,28,29,30
глобальный,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
нехватка,0.0,1.511883,1.511883,1.511883,1.511883,1.511883,0.0,0.0,0.0,0.0,0.0
микросхема,0.0,0.0,0.0,1.432702,0.0,0.0,0.0,0.0,2.865404,1.432702,1.432702
20202022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
год,0.0,0.0,0.648561,0.648561,0.0,0.648561,0.648561,1.945682,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
развязывание,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
кровопролитный,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
наслать,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
отображение,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# нормализация векторов документов

for i in df.columns:
    norm = np.sqrt((df[i]*df[i]).sum())
    for elem in unique:
        df.loc[elem, i] = df.loc[elem, i] / norm

In [22]:
df.head(25)

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
глобальный,0.319993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
нехватка,0.192602,0.0,0.129111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
микросхема,0.182515,0.0,0.0,0.0,0.140577,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20202022,0.319993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
год,0.082621,0.0,0.0,0.0,0.127273,0.0,0.081181,0.160199,0.0,0.100408,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
также,0.173986,0.0,0.116632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.244196,0.0,0.0,0.0,0.0,0.0,0.0
дефицит,0.23095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
кризис,0.461901,0.239023,0.0,0.125866,0.0,0.0,0.226923,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
полупроводник,0.198431,0.205367,0.133018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
который,0.123292,0.127601,0.0,0.134386,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.172801,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
mean = df.replace(0, np.nan)

In [24]:
mean_tf = mean.mean().mean()

In [25]:
print(mean_tf)

0.30609466783111217


In [26]:
print(data[7])

['остров', 'страдать', 'сам', 'сильный', 'засуха', '56', 'год', 'изз', 'количество', 'осадки', 'апрелемая', '2021', 'год', 'рекордно', 'низкий']


In [27]:
print(df[215].sum())

2.6334231821338974


In [28]:
print(sent_main[7].text)

Остров страдал от самой сильной засухи за 56 лет из-за того, что количество осадков в апреле-мае 2021 года было рекордно низким.


Необходимо нормализовать запрос и представить его в виде вектора в базисе unique.

In [55]:
Q1_main = 'Кризис микрочипов может продлиться от двух до 4 лет.'
Q2_main = 'Племена Эритреи общались между собой на «упрощённом итальянском».'
Q3_main = 'Экономические санкции стали инструментом геополитики ещё в V веке до н. э.'

In [56]:
def norm_and_vect(Q):
    Q1 = Q
    # удаление знаков препинания
    marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_—»«'''
    for x in Q1:  
        if x in marks:  
            Q1 = Q1.replace(x, "")
            
    Q1 = word_tokenize(Q1, language="russian")
    Q1 = filtered_words(Q1, stop_words)
    
    Q1_norm = []
    for word in Q1:
        Q1_norm.append(morph.parse(word)[0].normal_form)


    print(Q1_norm)

    Q1_vect = [0] * len(unique)
    idx = 0
    for elem in unique:
        w_cnt = 0
        for word in Q1_norm:
            if (word == elem):
                w_cnt += 1
        Q1_vect[idx] = w_cnt * idf_list[idx]
        idx += 1
        
    # нормализация запроса
    Q_tmp = [i ** 2 for i in Q1_vect]
    norm = np.sqrt(sum(Q1_vect))
    idx = 0
    for elem in Q1_vect:
        Q1_vect[idx] = elem / norm
        idx += 1
        
    return(Q1_vect)

In [57]:
Q1_vect = norm_and_vect(Q1_main)
Q2_vect = norm_and_vect(Q2_main)
Q3_vect = norm_and_vect(Q3_main)


['кризис', 'микрочип', 'продлиться', 'два', '4', 'год']
['племя', 'эритрея', 'общаться', 'себя', 'упрощённый', 'итальянский']
['экономический', 'санкция', 'стать', 'инструмент', 'геополитика', 'ещё', 'v', 'век', 'н', 'э']


In [59]:
print(sum(Q1_vect))

2.680337774198439


Для получения весов документов необходимо вычислить скалярное произведение векторизованного запроса с предложениями.

In [60]:
def output(Q, df):
    w_list = [0] * N
    idx = 0
    for i in df.columns:
        w = np.dot(df[i].to_list(), Q)
        # print(w)
        w_list[idx] = w
        idx += 1
    
    output_list = []
    idx = 0
    for val in w_list:
        if val != 0:
            output_list.append([val, idx])
        idx += 1
        
    output_Q = sorted(output_list, reverse = True)
    return output_Q

In [61]:
Q1_out = output(Q1_vect, df)
Q2_out = output(Q2_vect, df)
Q3_out = output(Q3_vect, df)

In [62]:
Q1_f_out = open('Q1_out.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q1_main)
Q1_f_out.write('Запрос: ' + Q1_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q1_out:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q1_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q1_f_out.close()

Запрос:  Кризис микрочипов может продлиться от двух до 4 лет.


In [63]:
Q2_f_out = open('Q2_out.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q2_main)
Q2_f_out.write('Запрос: ' + Q2_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q2_out:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q2_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q2_f_out.close()

Запрос:  Племена Эритреи общались между собой на «упрощённом итальянском».


In [64]:
Q3_f_out = open('Q3_out.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q3_main)
Q3_f_out.write('Запрос: ' + Q3_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q3_out:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q3_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q3_f_out.close()

Запрос:  Экономические санкции стали инструментом геополитики ещё в V веке до н. э.


*****Второй варинат выдачи*****

In [65]:
df_aug = pd.DataFrame(columns = np.arange(325), index = unique)

In [66]:
max_word = [0] * len(unique)
idx = 0
for elem in unique:
    max_w = 0
    for s in data:
        cur_cnt = 0
        for word in s:
            if (word == elem):
                cur_cnt += 1
        if (cur_cnt >= max_w):
            max_w = cur_cnt
    max_word[idx] = max_w
    idx += 1

In [67]:
for i in df_aug.columns:
    tmp = data[i]
    # print("tmp: ", tmp)
    elem_id = 0
    for elem in unique:
        tf_aug = 0
        for word in tmp:
            if (elem == word):
                tf_aug += 1
        if (tf_aug > 0):
            df_aug.loc[elem, i] = (0.5 + 0.5 * tf_aug / max_word[elem_id]) * idf_list[elem_id]
        else:
            df_aug.loc[elem, i] = 0
        elem_id += 1

In [68]:
# нормализация векторов документов

for i in df.columns:
    norm = np.sqrt((df_aug[i]*df_aug[i]).sum())
    for elem in unique:
        df_aug.loc[elem, i] = df_aug.loc[elem, i] / norm

In [69]:
df_aug.head(25)

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
глобальный,0.361537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
нехватка,0.217606,0.0,0.134593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
микросхема,0.154657,0.0,0.0,0.0,0.107713,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20202022,0.361537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
год,0.062232,0.0,0.0,0.0,0.054178,0.0,0.055631,0.067469,0.0,0.067127,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
также,0.196574,0.0,0.121584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.245945,0.0,0.0,0.0,0.0,0.0,0.0
дефицит,0.260934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
кризис,0.260934,0.186178,0.0,0.100596,0.0,0.0,0.174944,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
полупроводник,0.224192,0.213284,0.138666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
который,0.104474,0.09939,0.0,0.071604,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1314,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
print(df_aug[100].sum())

3.4615485523843557


In [76]:
def norm_and_vect_aug(Q):
    Q1 = Q
    # удаление знаков препинания
    marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_—»«'''
    for x in Q1:  
        if x in marks:  
            Q1 = Q1.replace(x, "")
            
    Q1 = word_tokenize(Q1, language="russian")
    Q1 = filtered_words(Q1, stop_words)
    
    Q1_norm = []
    for word in Q1:
        Q1_norm.append(morph.parse(word)[0].normal_form)


    print(Q1_norm)

    Q1_vect = [0] * len(unique)
    idx = 0
    for elem in unique:
        w_cnt = 0
        for word in Q1_norm:
            if (word == elem):
                w_cnt += 1
        if (w_cnt > 0):
            Q1_vect[idx] = 1 * idf_list[idx]
        else:
            Q1_vect[idx] = 0
        idx += 1
        
    # нормализация запроса
    Q_tmp = [i ** 2 for i in Q1_vect]
    norm = np.sqrt(sum(Q1_vect))
    idx = 0
    for elem in Q1_vect:
        Q1_vect[idx] = elem / norm
        idx += 1
        
    return(Q1_vect)

In [77]:
Q1_vect_aug = norm_and_vect_aug(Q1_main)
Q2_vect_aug = norm_and_vect_aug(Q2_main)
Q3_vect_aug = norm_and_vect_aug(Q3_main)

['кризис', 'микрочип', 'продлиться', 'два', '4', 'год']
['племя', 'эритрея', 'общаться', 'себя', 'упрощённый', 'итальянский']
['экономический', 'санкция', 'стать', 'инструмент', 'геополитика', 'ещё', 'v', 'век', 'н', 'э']


In [78]:
print(sum(Q1_vect_aug))

2.680337774198439


In [79]:
Q1_out_aug = output(Q1_vect_aug, df_aug)
Q2_out_aug = output(Q2_vect_aug, df_aug)
Q3_out_aug = output(Q3_vect_aug, df_aug)

In [81]:
Q1_f_out = open('Q1_out_aug.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q1_main)
Q1_f_out.write('Запрос: ' + Q1_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q1_out_aug:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q1_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q1_f_out.close()

Запрос:  Кризис микрочипов может продлиться от двух до 4 лет.


In [54]:
Q2_f_out = open('Q2_out_aug.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q2_main)
Q2_f_out.write('Запрос: ' + Q2_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q2_out_aug:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q2_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q2_f_out.close()

Запрос:  Племена Эритреи общались между собой на «упрощённом итальянском».


In [55]:
Q3_f_out = open('Q3_out_aug.txt', 'w', encoding = 'UTF-8')

print('Запрос: ', Q3_main)
Q3_f_out.write('Запрос: ' + Q3_main + '\n' + 'Выдача ограничена весом более 0.02' + '\n' + '\n')

for var in Q3_out_aug:
    if (var[0] >= 0.02):
        #print('Вес - ', var[0], '. Выдача - ', sent_main[var[1]].text, '\n')
        Q3_f_out.write('Вес - ' + str(var[0]) + '. Выдача - ' + sent_main[var[1]].text + '\n' + '\n')
        
Q3_f_out.close()

Запрос:  Экономические санкции стали инструментом геополитики ещё в V веке до н. э.
